<a href="https://colab.research.google.com/github/ashwath-tech/Spam-detector-using-neural-networks/blob/main/SpamDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow import keras
from keras import regularizers
import numpy as np
import spacy
import os
import pandas as pd
import kagglehub

path = kagglehub.dataset_download("uciml/sms-spam-collection-dataset")
files_in_dataset = os.listdir(path)
dataset_file_path = os.path.join(path, "spam.csv")
df = np.array(pd.read_csv(dataset_file_path, encoding='latin-1'))


Using Colab cache for faster access to the 'sms-spam-collection-dataset' dataset.


In [14]:
nlp = spacy.load("en_core_web_sm")
for i in range(df.shape[0]):
  doc = nlp(df[i,1])
  df[i,1] = " ".join([token.lemma_ for token in doc])

In [15]:
df[:,0][df[:,0] == "ham"] = 0
df[:,0][df[:,0] == "spam"] = 1

In [16]:
from sklearn.model_selection import train_test_split
x_train,  x_test, y_train, y_test = train_test_split(df[:,1], df[:,0], test_size = 0.2, random_state = 30)

In [17]:
vectorizer = CountVectorizer()
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [18]:
model = Sequential([
    tf.keras.Input(shape=(6843,)),
    Dense(units = 50, activation = "relu",  kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    Dense(units = 10, activation = "relu",  kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    Dense(units = 1, activation = "sigmoid"),
  ]
)

In [19]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 50)             │       342,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │           510 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 342,721 (1.31 MB)

 Trainable params: 342,721 (1.31 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01)
)

In [21]:
x_train.sort_indices()
model.fit(
    x_train, y_train.astype(int), epochs = 25
)

Epoch 1/25
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.5335
Epoch 2/25
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.2394
Epoch 3/25
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.1970
Epoch 4/25
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.1610
Epoch 5/25
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.1400
Epoch 6/25
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.1360
Epoch 7/25
140/140 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.1330
Epoch 8/25
140/140 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.1186
Epoch 9/25
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.1217
Epoch 10/25
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 0.1291
Epoch 11/25
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.1180
Epoch 12/25
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.1379
Epoch 13/25
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.1317
Epoch 14/25
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.1059
Epoch 15/25
140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step 

In [22]:
y_pred = model.predict(x_test)
y_pred[y_pred > 0.5] = 1
y_pred[y_pred <= 0.5] = 0

35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


In [23]:
from sklearn.metrics import classification_report
print(classification_report(y_test.astype(int), y_pred.astype(int)))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       970
           1       0.93      0.87      0.90       145

    accuracy                           0.97      1115
   macro avg       0.95      0.93      0.94      1115
weighted avg       0.97      0.97      0.97      1115



In [29]:
mail = input("enter ur email: ")
doc = nlp(mail)
mail = " ".join([token.lemma_ for token in doc])
mail = vectorizer.transform([mail,])
if model.predict(mail) > 0.5:
  print("spam")
else:
  print("not spam")

enter ur email:  Unstop Banner Hi Ashwath Srinivasan    Congrats, you're eligible for the Muthoot M George Higher Education Scholarship 2025.   	Eligibility: For B.Tech, MBBS, or B.Sc Nursing programs.  	Scholarship upto INR 2.4 Lakh.   Apply Now  Best Regards,  Team Unstop (formerly Dare2Compete)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
not spam
